# Technology focus: Visium HD

In this notebook we will present an overview of the plotting functionalities of the `spatialdata` framework, in the context of a Visium HD dataset.

## Challenges in plotting the data

Before showing the code, here is are some short considerations on the challenges that arise when handling and plotting Visium HD data, and how we provide methods to address them within the `spatialdata` framework.

### Data dimensions

Let's first have a quick recap of the data dimensions on hand.
- The Visium HD capture location area is 6.5mm x 6.5mm in size.
- The smallest capture locations (bins) are 2µm x 2µm, leading to up to around 10.5 millions bins per dataset.
- In the current dataset, around 20000 genes are considered.
- In the current dataset, only around 5.5 millions of the bins are actually covering parts of the tissue where gene expression is detected.

### Raster vs vector data representation

The bins are arranged in a 3250 x 3250 regular grid, so a natural way to represent the data would as an image. Nevertheless, due to large number of genes (and the data sparsity), using a purely raster representation is not feasible.
In fact, assuming using a `uint8` data type (1 byte) for recording, for each bin, a number from 0 to 255, rasterizing the full data would require 3250 x 3250 x 19059 = 187 GBs!

A more memory-efficient approach, that we implemented, is to decouple the spatial locations and the gene expression information:
- spatial locations are represented as vector geometries (circles) and are at most around 10.5 millions;
- gene expression information is stored as a `AnnData` table with sparse compressed representation (compressed sparse rows); in this dataset there are around 130 millions non-zero entries.

For the current dataset, assuming 1 byte per table entry, this would require around 130 MB of memory, much less than the upper limit 187 GB.

### The data representation is flexible

As we said above, we represent Visium HD data as a combination of circles and `AnnData` tables with a compressed sprase rows matrix. Nevertheless, there are cases where the user may have specific need for a data representation over the other; here is a list of examples of what `spaitaldata` offers.

- The compressed sparse rows matrix representation is efficient to access all the genes expressed in a particular bin; there are cases where one wants to go the other way around, and efficiently access all the locations expressing a particular gene. The `AnnData` API `tocsc()` allow to switch to a compressed sparse column repreentation.
- Plotting squares vs circles. Using squares is more accurate than using circles when plotting, as this reflects the shape of the catpure area; nevertheless, it is more performant to plot circles, reason why we adopt them by default. In this notebook we show how it is easy to switch between circles and squares. We use squares to plot "zoomed" version of the data.
- Raster vs vector representation. We explained above that a fully rasterized version of the data is not memory efficient; nevertheless if the user just want to visualize a small number of genes, the memory impact would be small. In such case, visualizing with raster objects is generally much more performant than visualizing shapes geometries. Thanks to the function `rasterize_bins()`, we can construct a lazy image that computes a rasterization on-demand, bridging the benefit of our lightweight vector data representationa and in-memory raster representation. As we will expain, the `rasterize_bins()` procudes a raster image that is aware of the coordinate transformations, so, no matter the bin size considered, the data is always aligned together with the high-resolution H&E image.

We will show all of this below.

In [1]:
%load_ext jupyter_black
%load_ext autoreload
%autoreload 2

## Loading the data

A reader for Visium HD data is available in `spatialdata-io`. We used it to parse and convert to Zarr a [Visium HD dataset of a Mouse Small Intestine (FFPE)](https://www.10xgenomics.com/datasets/visium-hd-cytassist-gene-expression-libraries-of-mouse-intestine).

We provide the already-converted Zarr data [available for download here](https://s3.embl.de/spatialdata/spatialdata-sandbox/visium_hd_3.0.0_io.zip).

Please download the data, rename the `.zarr` store to `visium_hd.zarr` and place it in the current folder (in alterantive you can use symlinks to make the data visible).

In [2]:
visium_hd_zarr_path = "./visium_hd.zarr"

A note on data loading. The data requires ~10 seconds to load because, while we support lazy representation of images, labels and points, the shapes geometries and the annotation tables are currently not represented lazily. This is one of the first spatial omics datasets which reaches a scale for which this is required. We will make a new release to allow for lazy representation also of these data types.

In [ ]:
%%time
import spatialdata as sd

sdata = sd.read_zarr(visium_hd_zarr_path)
sdata

The datasets contains 1 large microscopy image, represented as a multiscale, chunked image; two explicit downscaled versions of it and one CytAssist image.

Also, the image dataset contains the data at the highest resolution (2µm bins), and 2 downsampled (binned) versions of it, which have respectively bin sizes of 8µm and 16µm.

In [ ]:
# let's make the var names unique, this improves performance in accessing the tabular data
for table in sdata.tables.values():
    table.var_names_make_unique()

## Plotting the images

Let's visualize the images.

In [ ]:
import matplotlib.pyplot as plt
import spatialdata_plot

axes = plt.subplots(1, 2, figsize=(10, 5))[1].flatten()
sdata.pl.render_images("Visium_HD_Mouse_Small_Intestine_full_image").pl.show(ax=axes[0], title="Full image")
sdata.pl.render_images("Visium_HD_Mouse_Small_Intestine_cytassist_image").pl.show(ax=axes[1], title="CytAssit image")

Let's plot the same range for the 2 images; to achieve this we first compute the extent of the first image with `spatialdata.get_extent()` and then crop the second data using the `spatialdata` query APIs.

Please note that setting the plotting lim (`ax.set_xlim()`, ...) after plotting may lead to lower image quality because the data is plotted at the optimal resolution for the full extent but then a portion of it is zoomed in.

In [ ]:
from spatialdata import get_extent

data_extent = get_extent(sdata["Visium_HD_Mouse_Small_Intestine_full_image"], coordinate_system="global")
data_extent

In [ ]:
from spatialdata import bounding_box_query

queried_cytassist = bounding_box_query(
    sdata["Visium_HD_Mouse_Small_Intestine_cytassist_image"],
    min_coordinate=[data_extent["x"][0], data_extent["y"][0]],
    max_coordinate=[data_extent["x"][1], data_extent["y"][1]],
    axes=("x", "y"),
    target_coordinate_system="global",
)
sdata["queried_cytassist"] = queried_cytassist

In [ ]:
axes = plt.subplots(1, 2, figsize=(10, 5))[1].flatten()
sdata.pl.render_images("Visium_HD_Mouse_Small_Intestine_full_image").pl.show(ax=axes[0], title="Full image")
sdata.pl.render_images("queried_cytassist").pl.show(ax=axes[1], title="CytAssit image")

Let's focus the visualization on a smaller region, so we can appreciate better resolution of the first image. Here we create cropped version of the `SpatialData` objects on-the-fly using an anonymous function.

In [ ]:
axes = plt.subplots(1, 2, figsize=(10, 5))[1].flatten()
crop0 = lambda x: bounding_box_query(
    x, min_coordinate=[5000, 8000], max_coordinate=[10000, 13000], axes=("x", "y"), target_coordinate_system="global"
)
crop0(sdata).pl.render_images("Visium_HD_Mouse_Small_Intestine_full_image").pl.show(
    ax=axes[0], title="Full image", coordinate_systems="global"
)
crop0(sdata).pl.render_images("queried_cytassist").pl.show(
    ax=axes[1], title="CytAssit image", coordinate_systems="global"
)

## Plotting the gene expression data

### Plotting the geometries

Let's plot the bins colored by gene expression. For the moment we will use the largest bin size. Later in the notebook we will make plots using the two smaller bin sizes on a cropped version of the data. 

We will first plots the data as they are natively represented (circles), then show how to efficiently plot a rasterized version using `rasterize_bins()`.

In [ ]:
%%time
plt.figure(figsize=(10, 10))
ax = plt.gca()

gene_name = "AA986860"
# let's use method='matplotlib' to disable the default 'datashader' plotting since here we want to showcase the vector
# visualization. Slso datashader plots a rasterized version of the data that is not aware of the grid-like structure
# of the data, and therefore is not optimized for Visium HD data. For the optimized version we will show the usage of
# `rasterize_bins()` later.
sdata.pl.render_shapes("Visium_HD_Mouse_Small_Intestine_square_016um", color=gene_name, method="matplotlib").pl.show(
    coordinate_systems="global", ax=ax
)

### Performant on-the-fly data rasterization

Let's now pre-computed a lazy-rasterized version of the data using `rasterize_bins()`. This operation takes just a few seconds and unlocks very fast on-demand channel-wise rasterization.

In [ ]:
%%time
from spatialdata import rasterize_bins

for bin_size in ["016", "008", "002"]:
    # rasterize_bins() requires a compresed sparse column (csc) matrix
    sdata.tables[f"square_{bin_size}um"].X = sdata.tables[f"square_{bin_size}um"].X.tocsc()
    rasterized = rasterize_bins(
        sdata,
        f"Visium_HD_Mouse_Small_Intestine_square_{bin_size}um",
        f"square_{bin_size}um",
        "array_col",
        "array_row",
    )
    sdata[f"rasterized_{bin_size}um"] = rasterized

This produces lazy image objects that can be accessed gene-wise extremely efficiently. 

Importantly, these objects
**not** be computed as a whole, because this would lead to the unnecessary computation of hundreds of GB of memory.

In [ ]:
%%time
# this is very fast
sdata["rasterized_002um"].sel(c=gene_name).compute()
# this must not be called
# sdata["rasterized_002um"].compute()

Here is an example on how to plot the rasterized data. Please set `scale="full"`, this is essential. If it is not set, `spatialdata-plot` will try
to rerasterize the data to fit the canvas size and trying to compute the whole object.

In [ ]:
%%time
plt.figure(figsize=(10, 10))
ax = plt.gca()

sdata.pl.render_images("rasterized_016um", channel=gene_name, scale="full").pl.show(coordinate_systems="global", ax=ax)

As you can see below, we can plot the 2µm bins very performantly, which would not be possible with the `matplotlib` based approach shown before.

In [ ]:
%%time
import numpy as np
from matplotlib.colors import LinearSegmentedColormap

plt.figure(figsize=(10, 10))
ax = plt.gca()

# doesn't work due to this bug: https://github.com/scverse/spatialdata-plot/issues/303
# sdata.pl.render_images(
#     "rasterized_002um", channel=gene_name, scale="full", norm=matplotlib.colors.Normalize(vmin=0, vmax=4)
# ).pl.show(coordinate_systems="global", ax=ax)

# "flag" is a qualitative colormap that should not be used for anlaysis, here it's use to showcase the
# performance of plotting the rasterized data, viridis will be used as soon as the above bug is
# addressed
sdata.pl.render_images("rasterized_002um", channel=gene_name, scale="full", cmap="flag").pl.show(
    coordinate_systems="global", ax=ax
)

As a final not on the on-the-fly rasterization approach. Please, by looking at the corners of the last plot, notice how the data is on a grid that is actually sligthly rotated.
The advantage of using `rasterize_bins()` is that the produced object contains the coordinate transformations necessary to align (rotation and scale) the rasterized data together
with the high-resolution images.

### Plotting subsets of the data

Let's crop the data and make plots for all the bin sizes.

In [ ]:
sdata_small = sdata.query.bounding_box(
    min_coordinate=[7000, 11000], max_coordinate=[10000, 14000], axes=("x", "y"), target_coordinate_system="global"
)

In [ ]:
gene_name = "AA986860"
sdata_small.pl.render_shapes("Visium_HD_Mouse_Small_Intestine_square_016um", color=gene_name).pl.show(
    coordinate_systems="global"
)

The bins are represented as circles for performance reasons (`matplotlib` is efficient at both plotting circles and squares, but not `napari`). Let's quickly convert the circles to squares.

In [ ]:
from geopandas import GeoDataFrame
from shapely import Point, Polygon
from spatialdata.models import ShapesModel
from spatialdata.transformations import get_transformation


def square_from_circle(point: Point, r: float) -> Polygon:
    x, y = point.xy
    x = x[0]
    y = y[0]
    global rr
    rr = r
    return Polygon([(x - r, y - r), (x - r, y + r), (x + r, y + r), (x + r, y - r)])


# an optimized and more complete version of this function will be available in the spatialdata library in a next release
def squares_from_circles(gdf: GeoDataFrame) -> GeoDataFrame:
    geoseries = gdf[["geometry", "radius"]].apply(lambda row: square_from_circle(row[0], row[1]), axis=1)
    squares = GeoDataFrame(geometry=geoseries)
    transformations = get_transformation(gdf, get_all=True)
    return ShapesModel.parse(squares, transformations=transformations)

In [ ]:
for name in list(sdata_small.shapes.keys()):
    squares = squares_from_circles(sdata_small[name])
    sdata_small[name] = squares

In [ ]:
gene_name = "AA986860"

for bin_size in [16, 8, 2]:
    sdata_small.pl.render_shapes(f"Visium_HD_Mouse_Small_Intestine_square_{bin_size:03}um", color=gene_name).pl.show(
        coordinate_systems="global", title=f"bin_size={bin_size}µm", figsize=(10, 10)
    )

The data present a lot of sparsity, let's remake the plots above by visualizing only the non-zero entries, and using the full resolution image as a background.

We will do this by modifying the viridis colormap so that 0 is plotted as transparent. Let' also truncate the viridis colormap so that the highest value is colored as green and not yellow, since green has a better contrast against the pink of the H&E microscopy image.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import LinearSegmentedColormap

# modify the viridis colormap, so that the top color is a green (better visible on the H&E pink), and such that
# the value 0 leads to a transparent color
viridis = plt.get_cmap("viridis", 256)
# using 0.8 instead of 1.0 truncates the colormap
colors = viridis(np.linspace(0, 0.8, 256))
# set the color of zero to be transparent
colors[0, :] = [1.0, 1.0, 1.0, 0.0]

new_cmap = LinearSegmentedColormap.from_list("truncated_viridis", colors)

In [ ]:
gene_name = "AA986860"

for bin_size in [16, 8]:
    sdata_small.pl.render_images("Visium_HD_Mouse_Small_Intestine_full_image").pl.render_shapes(
        f"Visium_HD_Mouse_Small_Intestine_square_{bin_size:03}um", color=gene_name, cmap=new_cmap
    ).pl.show(coordinate_systems="global", title=f"bin_size={bin_size}µm", figsize=(10, 10))

Let's make a zoomed version of the plot for the 2µm bins, to better visiualize them. Please notice that we pass `method='matplotlib'` as we want to disable the default datashader rasterization, to have control on the colormap.

In [ ]:
crop1 = lambda x: bounding_box_query(
    x, min_coordinate=[9000, 11000], max_coordinate=[10000, 12000], axes=("x", "y"), target_coordinate_system="global"
)
crop1(sdata_small).pl.render_images("Visium_HD_Mouse_Small_Intestine_full_image").pl.render_shapes(
    "Visium_HD_Mouse_Small_Intestine_square_002um", color=gene_name, cmap=new_cmap, method="matplotlib"
).pl.show(coordinate_systems="global", title=f"bin_size=2µm", figsize=(10, 10))

As you can see the 8µm bins are convenient for looking at gene expression distribution from a broad perspective (same for the 16µm bins, where some resolution can be sacrificed in exchange for a faster visualization). On the other hand, the 2µm bins allow to precisely locate the expressed genes in the tissue.

## Plotting clusters

Let's now color the 16µm bins by cluster identity. Let's reuse the clusters `gene_expression_graphclust` computed from 10x Genomics and available together with the raw data from the 10x Genomics website.

In [ ]:
import os
from tempfile import TemporaryDirectory

import pandas as pd
import requests

# For convenience we rehost the single file containing the clusters we are interested in.
# Let's download it in a temporary directory and read it in a pandas DataFrame. The file is 2 MB.
clusters_file_url = "https://s3.embl.de/spatialdata/misc/visium_hd_mouse_intestine_16um_graphclust.csv"

with TemporaryDirectory() as tmpdir:
    path = os.path.join(tmpdir, "data.csv")
    response = requests.get(clusters_file_url)
    with open(path, "wb") as f:
        f.write(response.content)
    df = pd.read_csv(path)

In [ ]:
df.head(3)

In [ ]:
# let's convert the Cluster dtype from int64 to categorical since later we want the plots to use a categorical colormap
df["Cluster"] = df["Cluster"].astype("category")
df.set_index("Barcode", inplace=True)

In [ ]:
sdata["square_016um"].obs.head(3)

Let's merge the data.

In [ ]:
sdata["square_016um"].obs["Cluster"] = df["Cluster"]

Let's plot the clusters on one of the data crops we used before.

In [ ]:
# let convert circles to squares before making the plot
name = "Visium_HD_Mouse_Small_Intestine_square_016um"
sdata[name] = squares_from_circles(sdata[name])

In [ ]:
%%time
crop0(sdata).pl.render_images("Visium_HD_Mouse_Small_Intestine_full_image").pl.render_shapes(
    "Visium_HD_Mouse_Small_Intestine_square_016um", color="Cluster"
).pl.show(coordinate_systems="global", title=f"bin_size=016µm", figsize=(10, 10))

## Interactive visualization with napari

### Vector-based visualization

To conclude the example here is a screenshot of `napari-spatialdata` used to visualize the data on 16µm bins. Currently `napari`'s performance is not optimized for the visualization of large collections of polygonal data (we are working on improving this together with the `napari` developers); this warning is displayed as a tooltip is the user hovers the mouse above the warning symbols in the bottom left.

![image.png](attachments/napari_visium_hd.png)

### Raster-based visualization

To overcome the above limitations, you can proceed with the on-demand rasterization approach that we explained above, by simply selecting the `rasterized_002um`, `rasterized_008um` and `rasterized_002um` layers. Here are two animations showing this approach.

The first shows the equivalence between the vector shapes and the rasterized version, in this case using the 16µm bins data.

![image.png](attachments/napari_visium_hd_animation0.gif)

The second showcases the perfomane of the viewer on the 2µm bins data.

![image.png](attachments/napari_visium_hd_animation1.gif)